# Thermo Properties


### Reading Polynomial parameters and Calculating Thermodynamic parameters

The thermo class given will be used to calculate the thermodynamic parameters. Also, you'll need [thermoDataNASA-9.yaml].  This file contains two arrays for each species for two separate temperature ranges: a_lo is for $200<T<1000$ K and a_hi is for $1000<T<6000$ K. We have followed Cantera nomenclature (http://www.cantera.org/docs/sphinx/html/cti/species.html#the-nasa-9-coefficient-polynomial-parameterization).


$$\frac{C^0_p(T)}{R_u}=a_0T^{−2}+a_1T^{−1}+a_2+a_3T+a_4T^2+a_5T^3+a_6T^4$$

$$\frac{H^0(T)}{R_u T}=−a_0T^{−2}+a_1 \frac{lnT}{T}+a_2+\frac{a_3}{2}T+\frac{a_4}{3}T^2+\frac{a_5}{4}T^3+\frac{a_6}{5}T^4+\frac{a_7}{T}$$

$$\frac{s^0(T)}{R_u}=−\frac{a_0}{2}T^{−2}−a_1T^{−1}+a_2 lnT+a_3T+\frac{a_4}{2}T^2+\frac{a_5}{3}T^3+\frac{a_6}{4}T^4+a_8$$

where $R_u = 8.31446 J/mol/K$ is the universal gas constant.

Note that Gordon and McBride start polynomial coefficient numbering at 1 and the last coeffients for enthalpy and entropy are named respectively $a_7 = b_1$ and $a_8 = b_2$. (http://ntrs.nasa.gov/archive/nasa/casi.ntrs.nasa.gov/20020085330_2002141071.pdf)

In [1]:
# This cell of code give you an implementation of the above formulas.
import yaml
import numpy as np

class thermo:
    
    def __init__(self, species, MW) :
        """
        species: input string name of species in thermoData.yaml
        M: input (species molecular weight, kg/kmol)
        """
        
        self.Rgas = 8.31446      # J/mol*K
        self.M    = MW
    
        with open("thermoDataNASA-9.yaml") as yfile:
           yfile = yaml.safe_load(yfile)
        
        self.a_lo = yfile[species]["a_lo"]
        self.a_hi = yfile[species]["a_hi"]
        
        self.T_lo = 200.
        self.T_mid = 1000.
        self.T_hi = 6000.
        
    def cymalalo(self) :
        return self.a_lo

    def cymalahi(self) :
        return self.a_hi

    def cp_mole(self,T) :
        """
        return calorific value at cst p in units J/kmol/K
        T: input (K)
        """
        if T<=self.T_mid and T>=self.T_lo :
            a = self.a_lo
        elif T>self.T_mid and T<=self.T_hi :
            a = self.a_hi
        else :
            print ("ERROR: temperature is out of range")

        cp = a[0]/T**2 + a[1]/T + a[2] + a[3]*T + a[4]*T**2.0 + a[5]*T**3 + a[6]*T**4
        
        return cp * self.Rgas
        
    #--------------------------------------------------------

    def cp_mass(self,T) :
        """
        return calorific value at cst P in units of J/kg/K
        T: input (K)
        """
        return self.cp_mole(T)/self.M

    #--------------------------------------------------------
    
    def h_mole(self,T) :
        """
        return enthalpy in units of J/mol
        T: input (K)
        """
        if T<=self.T_mid and T>=self.T_lo :
            a = self.a_lo
        elif T>self.T_mid and T<=self.T_hi :
            a = self.a_hi
        else :
            print ("ERROR: temperature is out of range")

        hrt = -a[0]/T**2 + a[1]*np.log(T)/T + a[2] + a[3]/2*T + a[4]/3*T**2.0 + a[5]/4*T**3 + a[6]/5*T**4 + a[7]/T
        
        return hrt * self.Rgas * T
        
    #--------------------------------------------------------

    def h_mass(self,T) :
        """
        return enthalpy in units of J/kg
        T: input (K)
        """
        return self.h_mole(T)/self.M

    #--------------------------------------------------------
        
    def s_mole(self,T) :
        """
        return entropy in units of J/mol/K
        T: input (K)
        """
        if T<=self.T_mid and T>=self.T_lo :
            a = self.a_lo
        elif T>self.T_mid and T<=self.T_hi :
            a = self.a_hi
        else :
            print ("ERROR: temperature is out of range")
        
        sr = -a[0]/2/T**2 - a[1]/T + a[2]*np.log(T) + a[3]*T + a[4]/2.0*T**2.0 + a[5]/3.0*T**3.0+ a[6]/4.0*T**4.0+ a[8]
        
        return sr * self.Rgas
        
    #--------------------------------------------------------

    def s_mass(self,T) :
        """
        return entropy in units of J/kg/K
        T: input (K)
        """
        return self.s_mole(T)/self.M

In [2]:
# Example on how to use the class thermo to calculate thermodynamics 
# quantities for a specific species.
Molecule = "C8H18"    
MW_molecule = 114               # kg/kmol

t = thermo(Molecule,MW_molecule) # thermo object;
Temperature = 2000       #K
print('Enthalpy of ',Molecule,' at ',str(Temperature), 'K: ',t.h_mole(Temperature),' J/mol')
print('Entropy of ',Molecule,' at ',str(Temperature), 'K: ',t.s_mole(Temperature),' J/mol-K')
print('Specific heat of ',Molecule,' at ',str(Temperature), 'K: ',t.cp_mole(Temperature),' J/mol')


Enthalpy of  C8H18  at  2000 K:  552095.8600782972  J/mol
Entropy of  C8H18  at  2000 K:  1171.1651410939687  J/mol-K
Specific heat of  C8H18  at  2000 K:  593.5354818172497  J/mol


In [3]:
# If you wish to use this piece of code embedded in another code, you might need the following lines:
# Careful, you might need the following function the first time you are using # the code: "!pip install ipynb"
from ipynb.fs.full.ThermoPropertiesNASA import thermo

# Try it out in a new script, if it works, it should display the same results than the cell above
# If it works, just copy the content of the previous cell in your new script, and adapt it to your needs

ModuleNotFoundError: No module named 'ipynb'

In [ ]:
# Temperature = 300
# HHV = -1*thermo('C8H18',114).h_mole(Temperature) - 12.5*thermo('O2',32).h_mole(Temperature) + 8*thermo('CO2',44).h_mole(Temperature) + 9*(-285820)
# print(HHV)